In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from minority_report.clean_data import CleanData
from minority_report.scaling import Scaling
from datetime import datetime, timedelta
from geopandas import GeoSeries
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Load data

In [ ]:
df = Scaling()
df.load_data()

In [ ]:
sample = df.data[['period', 'latitude', 'longitude']]

In [ ]:
sample

In [ ]:
inf = sample['period'] > datetime(2013, 7, 1, 0, 0, 0)
sup = sample['period'] < datetime(2013, 8, 1, 0, 0, 0)

In [ ]:
interval = sample[inf & sup]
interval

In [ ]:
geometry_all = [Point(xy) for xy in zip(df.data['latitude'], df.data['longitude'])]
#geodataset = interval.drop(['longitude', 'latitude'], axis=1)
geo = GeoSeries(geometry_all)

In [ ]:
print(df.data['latitude'].min(), df.data['latitude'].max())

In [ ]:
print(df.data['longitude'].min(), df.data['longitude'].max())

In [ ]:
import geopandas
from geopandas import GeoDataFrame, GeoSeries
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
from shapely.geometry import Point, Polygon
import numpy as np
import googlemaps
from datetime import datetime
plt.rcParams["figure.figsize"] = [8,6]

fig, ax = plt.subplots()

# make a grid of latitude-longitude values
xmin, xmax, ymin, ymax = -74.21, -73.73, 40.530765, 40.887224
xx, yy = np.meshgrid(np.linspace(xmin,xmax,100), np.linspace(ymin,ymax,100))
xc = xx.flatten()
yc = yy.flatten()

# Now convert these points to geo-data
pts = GeoSeries([Point(x, y) for x, y in zip(xc, yc)])

# Plot to make sure it makes sense:
pts.plot(markersize=1, ax=ax)
geo.plot(ax=ax)

# Explore data

In [ ]:
from datetime import datetime

def group_by_hour(df, year, month, day):
    '''
    get a sample of a month-time crimes grouped by hour
    inputs = start_date info
    '''
    sample = df.data[['period', 'latitude', 'longitude']]
    
    inf = sample['period'] > datetime(year, month, day, 0, 0, 0)
    next_month = month+1
    next_year = year
    if month == 12:
        next_month = 1
        next_year = year+1
    print(next_year, next_month)
    sup = sample['period'] < datetime(next_year, next_month, day, 0, 0, 0)
    
    sample = sample[ inf & sup ]
    liste = np.sort(np.array(sample['period'].unique()))
    
    grouped = []
    length = len(liste)
    for index, timestamp in enumerate(liste):
        if (index+1) % 10 ==0:
            print(f'Grouping timestamp {index+1}/{length}')
        by_hour = np.array(sample[sample['period']== timestamp][['latitude', 'longitude']])
        grouped.append(by_hour) 
    lat_per_image = [[element[0] for element in crime] for crime in grouped]
    lon_per_image = [[element[1] for element in crime] for crime in grouped]
    return lat_per_image, lon_per_image

In [ ]:
lats, lons = group_by_hour(df, 2013, 8, 5)

In [ ]:
len(lons)

In [ ]:
len(lats)

In [ ]:
-74.134365483 - -73.958517839

In [ ]:
40.715107033 - 40.61867232

In [ ]:
lons

In [ ]:
lats

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(lats[0], lons[0])

In [ ]:
#latitude max-min de NYC
40.934109, -74.051895 (NW)
40.530765, -73.744835 (SE)

#faire un maillage de NYC
40.763934, -73.976322 to 

In [ ]:
(40.934109 - 40.530765)/-0.00045 #latitude

In [ ]:
(-73.673232--74.051895)/0.006179099999999771  #longitude

In [ ]:
from geopandas import GeoSeries
from shapely.geometry import Point
import matplotlib.pyplot as plt
geometry = [Point(xy) for xy in zip(lons[3], lats[3])]
#df_geopandas = sample.drop(['longitude', 'latitude'], axis=1)
gdf = GeoSeries(geometry)

In [ ]:
#BOUNDS
from shapely.geometry import Polygon
import numpy as np
points = gdf # for one image
xmin, ymax = 40.887224, -74.261419 #(NW) 
xmax, ymin = 40.530765, -73.744835 #(SE)

In [ ]:
#import geopandas
#from geopandas import GeoDataFrame, GeoSeries
#import matplotlib.pyplot as plt
#from matplotlib.colors import Normalize
#import matplotlib.cm as cm
#%matplotlib inline
#import seaborn as sns
#from shapely.geometry import Point, Polygon
#import numpy as np
#import googlemaps
#from datetime import datetime
#plt.rcParams["figure.figsize"] = [8,6]

# Get the shape-file for NYC
#boros = GeoDataFrame.from_file('../raw_data/geo_export_b18517e0-34fa-4296-ba66-861a5dbbd96d.sh')
#boros = boros.set_index('boro_code')
#boros = boros.sort_index()

# Plot and color by borough
#boros.plot(column = 'boro_name')

# Get rid of are that you aren't interested in (too far away)
#plt.gca().set_xlim([-74.05, -73.85])
#plt.gca().set_ylim([40.65, 40.9])

# make a grid of latitude-longitude values
#xmin, xmax, ymin, ymax = -74.05, -73.85, 40.65, 40.9
#xx, yy = np.meshgrid(np.linspace(xmin,xmax,100), np.linspace(ymin,ymax,100))
#xc = xx.flatten()
#yc = yy.flatten()

# Now convert these points to geo-data
#pts = GeoSeries([Point(x, y) for x, y in zip(xc, yc)])
#in_map =  np.array([pts.within(geom) for geom in boros.geometry]).sum(axis=0)
#pts = GeoSeries([val for pos,val in enumerate(pts) if in_map[pos]])

# Plot to make sure it makes sense:
#pts.plot(markersize=1)

In [ ]:
import geopandas
from geopandas import GeoDataFrame, GeoSeries
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
from shapely.geometry import Point, Polygon
import numpy as np
import googlemaps
from datetime import datetime
plt.rcParams["figure.figsize"] = [15, 15]

fig, ax = plt.subplots()

# make a grid of latitude-longitude values
xmin, xmax, ymin, ymax = -74.21, -73.73, 40.530765, 40.887224
xx, yy = np.meshgrid(np.linspace(xmin,xmax,100), np.linspace(ymin,ymax,100))
xc = xx.flatten()
yc = yy.flatten()

# Now convert these points to geo-data
pts = GeoSeries([Point(x, y) for x, y in zip(xc, yc)])

# Plot to make sure it makes sense:
pts.plot(markersize=1, ax=ax)
gdf.plot(ax=ax, color='black')
plt.gca().axison=False

#plt.savefig('../images/test.jpeg')
extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
fig.savefig('../images/test.jpeg', bbox_inches=extent)

In [ ]:
#from matplotlib.image import imread

#img = imread('../images/test.jpeg', 0)
#img.shape

In [ ]:
from skimage import io
img = io.imread('../images/test.jpeg')

In [ ]:
img.shape

In [ ]:
np.unique(img)

In [ ]:
plt.imshow(img)

# Get boundaries

In [2]:
import pandas as pd

In [3]:
boundaries = pd.read_csv('../raw_data/nybb.csv')

In [38]:
boundaries

,the_geom,BoroName,BoroCode,Shape_Leng,Shape_Area
0,MULTIPOLYGON (((-74.01092841268026 40.68449147...,Manhattan,1,361611.823950,6.366038e+08
1,MULTIPOLYGON (((-73.89680883223774 40.79580844...,Bronx,2,464650.051822,1.187245e+09
2,MULTIPOLYGON (((-73.86706149472118 40.58208797...,Brooklyn,3,729592.941749,1.934091e+09
3,MULTIPOLYGON (((-73.8264466151686 40.590527442...,Queens,4,900087.667215,3.044042e+09
4,MULTIPOLYGON (((-74.05050806403247 40.56642203...,Staten Island,5,325956.009000,1.623757e+09


In [39]:
boundaries['the_geom'][0]

'MULTIPOLYGON (((-74.01092841268026 40.68449147254294, -74.01193259977079 40.683887749154934, -74.01217596614636 40.684095185628465, -74.01011625533792 40.68534159773662, -74.0087859013092 40.686146602298905, -74.00869559889541 40.686193318012684, -74.00859803329713 40.686252564084974, -74.00835446532174 40.68640020025069, -74.00816414593905 40.686174717161464, -74.00842516151924 40.68601553244514, -74.0085129425734 40.685962548583696, -74.00860072436832 40.685909564654835, -74.00953293674742 40.68534164753917, -74.01092841268026 40.68449147254294)), ((-74.0050037331507 40.68760598489827, -74.00562986330391 40.68678420554105, -74.00783293766679 40.687385055162764, -74.00742012154092 40.6882062904359, -74.0050037331507 40.68760598489827)), ((-74.0038203891973 40.68892964419677, -74.00459270521766 40.68821522298663, -74.00678438439806 40.688826879958285, -74.00636461225261 40.689669668636974, -74.0038203891973 40.68892964419677)), ((-74.0029756543656 40.69042657592072, -74.00340921624513

In [ ]:
bounds = boudaries['the_geom'][0].replace('MULTIPOLYGON','')\
                                .replace('(','')\
                                .replace(')','')\
                                .strip().\
                                replace(' ',',')\
                                .split(',')
bound

In [ ]:
coords = [float(element) for element in bound if element != '']
coords

In [ ]:
latitude = []
longitude = []
for element in coords:
    if element > 0:
        latitude.append(element)
    else:
        longitude.append(element)
print(min(latitude), max(latitude))
print(min(longitude), max(longitude))

In [46]:
def get_local_min_max(coords):
    
    latitude = []
    longitude = []
    for element in coords:
        if element > 0:
            latitude.append(element)
        else:
            longitude.append(element)
    min_max_lat = [min(latitude), max(latitude)]
    min_max_lon = [min(longitude), max(longitude)]
    return min_max_lat, min_max_lon

In [33]:
def compare_lon_lat(lat, lon, best_lat, best_lon):
    #print(lat, best_lat)
    if lat[0] < best_lat[0]:
        best_lat[0] = lat[0]
    if lat[1] > best_lat[1]:
        best_lat[1] = lat[1]
    if lon[0] < best_lon[0]:
        best_lon[0] = lon[0]
    if lon[1] > best_lon[1]:
        best_lon[1] = lon[1]
    return best_lat, best_lon

In [44]:
def absolute_min_max(boundaries):
    best_lat = [0,0]
    best_lon = [0,0]
    for borough in range(boundaries.shape[0]):
        bounds = boundaries['the_geom'][borough].replace('MULTIPOLYGON','')\
                                .replace('(','')\
                                .replace(')','')\
                                .strip().\
                                replace(' ',',')\
                                .split(',')
        coords = [float(element) for element in bounds if element != '']
        lat, lon = get_local_min_max(coords)
        #import ipdb ; ipdb.set_trace()
        if (best_lat == [0,0]) & (best_lon ==[0,0]):
            best_lat = lat
            best_lon = lon
        else:
            best_lat, best_lon = compare_lon_lat(lat, lon, best_lat, best_lon)
    return best_lat, best_lon

In [47]:
absolute_min_max(boundaries)

([40.49611539518921, 40.91553277600008],
 [-74.25559136315213, -73.70000906387347])